# Panel for Sequential Applications

Panel, which is maintained by HoloViz (maintainers of HoloViews as well), creates a sequential User Interface with ```Pipeline``` objects.

In [1]:
import param # Needed for creating stages as panel is built off of param
import panel as pn
pn.extension('katex') # For mathematical notation, irrelevant library for general purpose

pipeline = pn.pipeline.Pipeline() # Instantiates an empty pipeline

Apparently, every time you create a stage class, it is a child of the ```param.Parameterized``` class. Additionally, when you want to add the stage class to the pipeline, you define the ```panel``` function to setup the layout of the stage. Everything before then is a widget of some sort.

In [2]:
class Stage1(param.Parameterized):

    a = param.Integer(default=2, bounds=(0, 10))
    b = param.Integer(default=3, bounds=(0, 10))

    @param.output(('c', param.Integer), ('d', param.Integer))
    def output(self):
        return self.a * self.b, self.a ** self.b

    @param.depends('a', 'b')
    def view(self):
        c, d = self.output()
        c_out = pn.pane.LaTeX('${a} * {b} = {c}$'.format(
            a=self.a, b=self.b, c=c), styles={'font-size': '2em'})
        d_out = pn.pane.LaTeX('${a}^{{{b}}} = {d}$'.format(
            a=self.a, b=self.b, d=d), styles={'font-size': '2em'})
        return pn.Column(
		    c_out, d_out,  margin=(40, 10), styles={'background': '#f0f0f0'}
		)

    def panel(self):
        return pn.Row(self.param, self.view,)

In [3]:
class Stage2(param.Parameterized):

    c = param.Integer(default=6, bounds=(0, None))
    exp = param.Number(default=0.1, bounds=(0, 3))

    @param.depends('c', 'exp')
    def view(self):
        out = pn.pane.LaTeX('${%s}^{%s}={%.3f}$' % (self.c, self.exp, self.c**self.exp),
                      styles={'font-size': '2em'})
        return pn.Column(out, margin=(40, 10), styles={'background': '#f0f0f0'})

    def panel(self):
        return pn.Row(self.param, self.view)

In [4]:
pipeline.add_stage('Stage 1', Stage1)
pipeline.add_stage('Stage 2', Stage2)

# pipeline

## ipywdigets

The ```ipywidgets``` library can be used in addition to everything in the panel package by plugging in Python widgets into the ```panel``` function. This can be placed inside of a ```panel``` library container like:

```python
def panel(self):
    return pn.Row(self.grid)
```

but, as you can see in the example, this is not strictly necessary

In [5]:
import ipywidgets as ipw

In [6]:
class Stage3(param.Parameterized):
    
    def create_expanded_button(description, button_style):
        return ipw.Button(description=description, button_style=button_style, layout=ipw.Layout(height='auto', width='auto'))
    
    grid = ipw.GridspecLayout(4, 3)

    for i in range(4):
        for j in range(3):
            grid[i, j] = create_expanded_button('Button {} - {}'.format(i, j), 'warning')
            
    def panel(self):
        return self.grid

In [7]:
pipeline.add_stage('Stage 3', Stage3)

# pipeline

## Bokeh

Similarly, Bokeh plots are available and they can be wrapped in ```panel``` layouts like:

```python
def panel(self):
    return pn.pane.Bokeh(p)
```

but, again, this is not strictly necessary.

In [8]:
from bokeh.models import BoxEditTool, ColumnDataSource
from bokeh.plotting import figure, show

In [9]:
class Stage4(param.Parameterized):
    p = figure(x_range=(0, 10), y_range=(0, 10), width=400, height=400,
           title='Box Edit Tool')

    src = ColumnDataSource({
        'x': [5, 2, 8], 'y': [5, 7, 8], 'width': [2, 1, 2],
        'height': [2, 1, 1.5], 'alpha': [0.5, 0.5, 0.5]
    })

    r = p.rect('x', 'y', 'width', 'height', source=src, alpha='alpha')

    draw_tool = BoxEditTool(renderers=[r], empty_value=1)
    p.add_tools(draw_tool)
    p.toolbar.active_drag = draw_tool

    def panel(self):
        return p

In [10]:
pipeline.add_stage('Stage 4', Stage4)

pipeline

Pipeline:
    [0] Stage 1: Stage1()
    [1] Stage 2: Stage2()
    [2] Stage 3: Stage3()
    [3] Stage 4: Stage4()